In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
player_ids = []

In [ ]:
driver = webdriver.Firefox()

In [ ]:
url = r"https://baseballsavant.mlb.com/visuals/statcast-pitch-highlighter?playerId=660670&type=&batHand=&pitchHand=&pitchType=&year=2023&zone="

In [ ]:
driver.get(url)

In [ ]:
select = Select(driver.find_element(By.XPATH,r"/html/body/div[2]/div/div/div[3]/div[1]/form/div[4]/div/select"))

In [ ]:
src = driver.page_source
parser = BeautifulSoup(src, "lxml")
table = parser.find("div", attrs = {"id": "calculations"})
headers = table.findAll('th')
headerlist = [h.text.strip() for h in headers[:]]

In [ ]:
select.select_by_index(0)

In [ ]:
pitchType = select.first_selected_option.text

In [ ]:
print(pitchType)

In [ ]:
headerlist[0] = "Name"

In [ ]:
headerlist

In [ ]:
stats = pd.DataFrame(columns=headerlist)

In [ ]:
stats

In [ ]:
rows = table.findAll('tr')[1:]

In [ ]:
# player_stats = [[td.getText().strip() for td in rows[i].findAll('td')] for i in range(len(rows))]

In [ ]:
player_stats = [[f"{pitchType}" if td.getText().strip() == '' else td.getText().strip() for td in rows[i].findAll('td')] for i in range(len(rows))]

In [ ]:
player_stats[0][0]

In [ ]:
len(headerlist)

In [ ]:
player_stats

In [ ]:
for i in range(len(player_stats)):
    stats.loc[i] = player_stats[0]

In [ ]:
stats

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd

player_ids = []
timeout = 25
mlb_seasons = ['2023', '2022', '2021']

In [2]:
driver = webdriver.Firefox()

In [3]:
url = r"https://baseballsavant.mlb.com/visuals/statcast-pitch-highlighter?playerId=660670&type=&batHand=&pitchHand=&pitchType=&year=2023&zone="
driver.get(url)

element_present = EC.presence_of_element_located((By.XPATH, r'//*[@id="ddlPitchTypes"]'))
WebDriverWait(driver, timeout).until(element_present)

headers_present = EC.presence_of_element_located((By.XPATH, r'//*[@id="calculations"]/table/thead/tr'))
WebDriverWait(driver, timeout).until(headers_present)

select = Select(driver.find_element(By.XPATH, r'//*[@id="ddlPitchTypes"]'))
select.select_by_index(0)
pitchType = select.first_selected_option.text
src = driver.page_source
parser = BeautifulSoup(src, "lxml")
table = parser.find("div", attrs = {"id": "calculations"})
headers = table.findAll('th')
headerlist = [h.text.strip() for h in headers[:]]
headerlist[0] = "Name"



In [ ]:
# rows = table.findAll('tr')[1:]
# player_stats = [[f"{pitchType}" if td.getText().strip() == '' else td.getText().strip() for td in rows[i].findAll('td')] for i in range(len(rows))]

In [4]:
stats = pd.DataFrame(columns=headerlist)

In [5]:
for season in mlb_seasons:
    for i in range(len(select.options)):
        url = f"https://baseballsavant.mlb.com/visuals/statcast-pitch-highlighter?playerId=660670&type=&batHand=&pitchHand=&pitchType=&year={season}&zone="
        driver.get(url)
        
        element_present = EC.presence_of_element_located((By.XPATH, r'//*[@id="ddlPitchTypes"]'))
        WebDriverWait(driver, timeout).until(element_present)

        data_present = EC.presence_of_element_located((By.XPATH, r'//*[@id="calculations-tr_0"]'))
        WebDriverWait(driver, timeout).until(data_present)
    
        select = Select(driver.find_element(By.XPATH, r'//*[@id="ddlPitchTypes"]'))
        select.select_by_index(f"{i}")
        pitchType = select.first_selected_option.text
        src = driver.page_source
        parser = BeautifulSoup(src, "lxml")
        table = parser.find("div", attrs = {"id": "calculations"})
        headers = table.findAll('th')
        headerlist = [h.text.strip() for h in headers[:]]
        headerlist[0] = "Name"
        rows = table.findAll('tr')[1:]
        player_stats = [[td.getText().strip() for td in rows[i].findAll('td')] for i in range(len(rows))]
        if player_stats[0][4] == "": player_stats[0][4] = pitchType
        if player_stats[0][1] == "": player_stats[0][1] = season

        # for row in range(len(player_stats)):
        #     stats.loc[i] = player_stats[row]

        stats2 = pd.DataFrame(player_stats, columns=headerlist)
        stats = pd.concat([stats,stats2], ignore_index=True)
        
    



In [6]:
stats

,Name,Season,Total Pitches,Pitches,Pitch Types,%,PA,AB,H,1B,...,BA,SO,Swings,Misses,Sw%,Whiff%,K%,wOBA,EV,Spin Rate
0,Acuna,2023,2782,2782,All Pitches,100.0,732,643,217,137,...,.337,84,1286,241,46.2,18.7,11.5,.428,94.4,2232
1,Acuna,2023,2782,1611,Fastballs,57.9,435,374,127,86,...,.340,46,759,111,47.1,14.6,10.6,.426,94.5,2247
2,Acuna,2023,2782,306,Offspeed,11.0,72,65,19,13,...,.292,11,142,38,46.4,26.8,15.3,.367,92.7,1651
3,Acuna,2023,2782,860,Breaking,30.9,224,203,71,38,...,.350,26,384,92,44.7,24.0,11.6,.452,94.6,2412
4,Acuna,2023,2782,242,Changeup,8.7,57,50,15,11,...,.300,7,109,27,45.0,24.8,12.3,.363,93.1,1739
5,Acuna,2023,2782,142,Curveball,5.1,41,35,14,8,...,.400,5,60,11,42.3,18.3,12.2,.539,98.7,2463
6,Acuna,2023,2782,5,Eephus,0.2,2,2,2,2,...,1.000,0,4,2,80.0,50.0,0.0,.883,100.3,1301
7,Acuna,2023,2782,3,BP Fastball,0.1,1,1,0,0,...,.000,1,1,0,33.3,0.0,100.0,.000,--,2102
8,Acuna,2023,2782,161,Cutter,5.8,40,37,17,14,...,.459,3,84,16,52.2,19.0,7.5,.492,90.4,2361
9,Acuna,2023,2782,855,4-Seam Fastball,30.7,206,174,53,27,...,.305,24,386,67,45.1,17.4,11.7,.423,95.4,2289
